<a href="https://colab.research.google.com/github/zhaochuninhefei/ipynbs/blob/master/ChatGLM_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 一、准备工作

## 1.1 选择GPU
在菜单`代码执行程序`中选择`更改运行时类型`,选择GPT，免费的T4。

然后检查当前N卡配置:

In [1]:
!nvidia-smi

Fri May 26 06:49:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

可以看到免费的T4显卡有15G显存

## 1.2 下载ChatGLM-6B项目
默认会下载到`/content/ChatGLM-6B`目录:

In [2]:
!git clone https://github.com/THUDM/ChatGLM-6B.git

Cloning into 'ChatGLM-6B'...
remote: Enumerating objects: 1176, done.
remote: Counting objects: 100% (612/612), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1176 (delta 526), reused 542 (delta 501), pack-reused 564
Receiving objects: 100% (1176/1176), 9.01 MiB | 29.01 MiB/s, done.
Resolving deltas: 100% (695/695), done.


## 1.3 安装依赖库
根据ChatGLM-6B项目目录下的`requirements.txt`文件安装相关依赖包。

In [3]:
!pip install -r /content/ChatGLM-6B/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 

# 二、加载模型
这里选择`chatglm-6b-int4`量化模型。
> 默认的`chatglm-6b`(FP16无量化模型)和int8量化模型都因为这里内存RAM不足(只有12.7G)而无法成功加载模型。

In [4]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4",trust_remote_code=True).half().cuda()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c
Compiling gcc -O3 -fPIC -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Using quantization cache
Applying quantization to glm layers


# 三、运行推理
## 3.1 首次推理
首次推理会慢一点。这里的提示词很简单，就是`你好`。

In [5]:
%%time
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。
CPU times: user 8 s, sys: 665 ms, total: 8.66 s
Wall time: 15.7 s


可以看到，首次运行推理花了15.4秒。

## 3.2 再次推理
然后我们再次运行相同提示词的推理：

In [6]:
%%time
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。
CPU times: user 4.8 s, sys: 0 ns, total: 4.8 s
Wall time: 4.8 s


快了不少，只要5.17秒，这个其实还是很慢。

## 3.3 其他推理
在这里我们尝试一下其他稍微复杂点的提示词:

In [7]:
%%time
response, history = model.chat(tokenizer, "桌子上有4个苹果，小红吃了1个，小刚拿走了2个，还剩下几个苹果？", history=[])
print(response)

还剩下 3 个苹果。
CPU times: user 1.69 s, sys: 0 ns, total: 1.69 s
Wall time: 1.69 s


这种计算推理问题，ChatGLM-6B的表现不是很好。

再尝试一个常识类问题:

In [8]:
%%time
response, history = model.chat(tokenizer, "中国四大名著是哪四本书?", history=[])
print(response)

中国四大名著是《红楼梦》、《西游记》、《水浒传》和《三国演义》。这四本书都是中国文学史上的经典之作，被广泛传诵和阅读，对中国文化产生了深远的影响。其中，《红楼梦》是中国文学史上的经典之作，被誉为“中国小说的经典之作”，也是世界文学史上的经典之作。《西游记》、《水浒传》和《三国演义》则分别被誉为“中国文学的三部曲”、“中国小说的三部曲”和“中国小说的四大名著”。
CPU times: user 16.5 s, sys: 0 ns, total: 16.5 s
Wall time: 16.6 s


这个还行。

## 3.4 信息提取
再尝试一个做信息提取的例子:

In [9]:
%%time
content="""本人是毕业于南七技校，性格开朗，待人真诚，对待工作认真负责，善于沟通、协调有较强的组织能力与团队精神;
活泼开朗、乐观上进、有爱心并善于施教并行;上进心强、勤于学习能不断提高自身的能力与综合素质。
在未来的工作中，我将以充沛的精力，刻苦钻研的精神来努力工作，稳定地提高自己的工作能力，与公司同步发展。
目前我居住在合肥市云深路234号奋斗小区，我的手机号码是: 18012345678。也可以用邮箱联系我: testtest@test.com。
"""
prompt="""从上文中，提取"信息"(keyword:content)，包括:"手机号"、"邮箱"、"毕业院校"、"住址"、"自我评价"等keyword及其content，输出json格式内容。
"""
input ='{}\n\n{}'.format(content,prompt)
print(input)
response, history = model.chat(tokenizer, input, history=[])
print(response)

本人是毕业于南七技校，性格开朗，待人真诚，对待工作认真负责，善于沟通、协调有较强的组织能力与团队精神;
活泼开朗、乐观上进、有爱心并善于施教并行;上进心强、勤于学习能不断提高自身的能力与综合素质。
在未来的工作中，我将以充沛的精力，刻苦钻研的精神来努力工作，稳定地提高自己的工作能力，与公司同步发展。
目前我居住在合肥市云深路234号奋斗小区，我的手机号码是: 18012345678。也可以用邮箱联系我: testtest@test.com。


从上文中，提取"信息"(keyword:content)，包括:"手机号"、"邮箱"、"毕业院校"、"住址"、"自我评价"等keyword及其content，输出json格式内容。

{  
 "手机号": 18012345678,  
 "邮箱": testtest@test.com,  
 "毕业院校": 南七技校，  
 "住址": 合肥市云深路234号奋斗小区，  
 "自我评价": 活泼开朗、乐观上进、有爱心并善于施教并行；上进心强、勤于学习能不断提高自身的能力与综合素质  
}
CPU times: user 17.6 s, sys: 2.12 ms, total: 17.6 s
Wall time: 17.7 s


这个效果还是不错的，这里可以看出来很明显的语境内学习能力。

## 3.5 信息统计
再来试一个信息收集统计：

In [26]:
%%time
content="""河马: 啦啦啦啦啦啦啦

张三: 怎么回事？突然断电了？

李四: 我家也断电了

王五: +1

傻大个: 没有啊，我家正常的啊

胖君子: 啊这，好累。

张三: @管家，什么情况？什么时候能来电? 

王五: 同问

彼岸花: 我家也停电了

兔子: +1

毛熊: +1

小鹰: 没停电的飘过。。。

管家: 正在联系供电局，请大家少安勿躁。

李四: 最近经常停电啊，世界末日要来了，大家出来嗨吧～～～

屁屁: 还好我家没停。

谁家大爷: 又停电了

胡萝卜: +1

管家: 正在联系供电局，请大家稍安勿躁。

"""
prompt='阅读上面的对话，请问有几家停电或断电了?给出数字即可。'
input ='{}\n\n{}'.format(content,prompt)
print(input)
response, history = model.chat(tokenizer, input, history=[])
print()
print("回答:")
print(response)

河马: 啦啦啦啦啦啦啦

张三: 怎么回事？突然断电了？

李四: 我家也断电了

王五: +1

傻大个: 没有啊，我家正常的啊

胖君子: 啊这，好累。

张三: @管家，什么情况？什么时候能来电? 

王五: 同问

彼岸花: 我家也停电了

兔子: +1

毛熊: +1

小鹰: 没停电的飘过。。。

管家: 正在联系供电局，请大家少安勿躁。

李四: 最近经常停电啊，世界末日要来了，大家出来嗨吧～～～

屁屁: 还好我家没停。

谁家大爷: 又停电了

胡萝卜: +1

管家: 正在联系供电局，请大家稍安勿躁。



阅读上面的对话，请问有几家停电或断电了?给出数字即可。

回答:
有5家停电或断电了。
CPU times: user 1.98 s, sys: 6.66 ms, total: 1.99 s
Wall time: 1.99 s


很遗憾，统计结果不正确，实际上应该是8家停电。